In [16]:
import os
import numpy as np
import cv2 as cv
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import h5py

from models.resnet_custom import resnet50_baseline
from utils.utils import print_network, collate_features

In [17]:
patch_dir = "image_sets/patches/"
feat_dir = "image_sets/features/"
actual_feat_dir = "image_sets/patches/fungal_vs_nonfungal_resnet_features/pt_files/"

In [18]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Create feat_dir if not exists. 
# Not properly fixed
if not os.path.exists(feat_dir):
    try:
        print("Features directory doesn't exist. Creating ...")
        os.mkdir(feat_dir, exist_ok=True)
    except:
        print("ERROR: Cannot create the Features directory")

model = resnet50_baseline(pretrained=True)
model = model.to(device)

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model.eval()

ResNet_Baseline(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck_Baseline(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(

In [19]:
# Create dataset from the image patches
dataset = []
for folder in sorted(os.listdir(patch_dir)):
    patch_folder = os.path.join(patch_dir, folder)
    for patch_file in sorted(os.listdir(patch_folder)):
        if patch_file == "pt_files":
            continue
        
        img_path = os.path.join(patch_folder, patch_file)

        img = Image.open(img_path)

        img_arr = np.asarray(img)
        # img_arr = np.expand_dims(img_arr, 0)
        # img_PIL = Image.fromarray(img_arr)

        # Create the dataset loader
        imgs = torch.tensor(img_arr)

        # Get coord in [x, y] format
        coord = img_path.split("/")
        coord = coord[-1]
        coord = coord.split(".")[-2]
        coord = coord.split("_")
        coord = [int(coord[-2])/256, int(coord[-1])/256]

        dataset.append([imgs, coord])

loader = DataLoader(dataset=dataset, batch_size=1)

In [20]:
patch_folders = [os.path.join(patch_dir, folder) for folder in sorted(os.listdir(patch_dir))]
print(patch_folders)
patches_per_image = len(patch_folders[0])
print(patches_per_image)

['image_sets/patches/F005a02', 'image_sets/patches/F006a01', 'image_sets/patches/F006a02', 'image_sets/patches/F006a03', 'image_sets/patches/F006a04', 'image_sets/patches/F006a05', 'image_sets/patches/F006a06', 'image_sets/patches/F006a07', 'image_sets/patches/F006a08', 'image_sets/patches/F006a09', 'image_sets/patches/F006a10', 'image_sets/patches/F007a01', 'image_sets/patches/F007a02', 'image_sets/patches/F007a03', 'image_sets/patches/F007a04', 'image_sets/patches/F007a05', 'image_sets/patches/F007a06', 'image_sets/patches/F007a07', 'image_sets/patches/F007a08', 'image_sets/patches/F007a09', 'image_sets/patches/F007a10', 'image_sets/patches/F007a11', 'image_sets/patches/F007a12', 'image_sets/patches/F007a13', 'image_sets/patches/F007a15', 'image_sets/patches/F007a16', 'image_sets/patches/F007a17', 'image_sets/patches/F007a18', 'image_sets/patches/F007a19', 'image_sets/patches/F007a21', 'image_sets/patches/F007a22', 'image_sets/patches/F009a01', 'image_sets/patches/F009a02', 'image_se

In [21]:
for count, data in enumerate(loader):
    with torch.no_grad():
        filename = str(patch_folders[count//patches_per_image])
        filename = filename.split("/")[-1]
        coord = data[1]
        batch = data[0]
        batch = torch.unsqueeze(batch, 0)
        batch = batch.reshape([1, 3, 256, 256])
        batch = batch.to(device, non_blocking=True)
        batch = batch.float()

        features = model(batch)
        features = features.cpu().numpy()
        features = torch.from_numpy(features)

        print(filename)
        filePath = os.path.join(feat_dir, filename+'.pt')
        print(count, " || ", coord, " || ", features, " || ", filePath)
        # print("Features size: ", features.shape)
        torch.save(features, filePath)

        # Save the .hdf5
        # hf = h5py.File('data.h5', 'w')

        print("="*15)

F005a02
0  ||  [tensor([0.], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.4815,  0.0000,  2.4632,  ..., 30.7341,  0.0000,  0.0000]])  ||  image_sets/features/F005a02.pt
F005a02
1  ||  [tensor([0.], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.4938,  0.0000,  2.5673,  ..., 30.4145,  0.0000,  0.0000]])  ||  image_sets/features/F005a02.pt
F005a02
2  ||  [tensor([0.], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.5683,  0.0000,  2.3696,  ..., 30.8572,  0.0000,  0.0000]])  ||  image_sets/features/F005a02.pt
F005a02
3  ||  [tensor([0.], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.4113,  0.0000,  1.8413,  ..., 29.8704,  0.0000,  0.0000]])  ||  image_sets/features/F005a02.pt
F005a02
4  ||  [tensor([0.], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.4613,  0.0000,  2.4482,  ..., 30.0833,  0.0000,  0.0000]])  ||  image_sets/features/F005a

F006a02
65  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.3165,  0.0000,  0.8949,  ..., 26.8774,  0.0000,  0.0000]])  ||  image_sets/features/F006a02.pt
F006a02
66  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.3118,  0.0000,  0.8134,  ..., 27.1647,  0.0000,  0.0000]])  ||  image_sets/features/F006a02.pt
F006a02
67  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.3851,  0.0000,  1.0684,  ..., 22.6928,  0.0000,  0.0000]])  ||  image_sets/features/F006a02.pt
F006a02
68  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.5062,  0.0000,  2.4706,  ..., 20.2973,  0.0000,  0.0000]])  ||  image_sets/features/F006a02.pt
F006a02
69  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.4083,  0.0000,  1.1763,  ..., 26.1106,  0.0000,  0.0000]])  ||  

F006a03
103  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.3502,  0.0000,  0.9189,  ..., 29.5747,  0.0000,  0.0000]])  ||  image_sets/features/F006a03.pt
F006a04
104  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.3645,  0.0000,  1.3692,  ..., 29.0219,  0.0000,  0.0000]])  ||  image_sets/features/F006a04.pt
F006a04
105  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.4395,  0.0000,  2.6697,  ..., 26.4151,  0.0000,  0.0000]])  ||  image_sets/features/F006a04.pt
F006a04
106  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.4284,  0.0000,  1.5474,  ..., 27.8037,  0.0000,  0.0000]])  ||  image_sets/features/F006a04.pt
F006a04
107  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.3320,  0.0000,  0.9661,  ..., 28.5794,  0.0000,  0.0000

F006a06
169  ||  [tensor([0.], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.4974,  0.0000,  2.0555,  ..., 29.8188,  0.0000,  0.0000]])  ||  image_sets/features/F006a06.pt
F006a06
170  ||  [tensor([0.], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.5490,  0.0000,  2.1470,  ..., 29.5662,  0.0000,  0.0000]])  ||  image_sets/features/F006a06.pt
F006a06
171  ||  [tensor([0.], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.6072,  0.0000,  2.7209,  ..., 28.4821,  0.0000,  0.0000]])  ||  image_sets/features/F006a06.pt
F006a06
172  ||  [tensor([0.], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.7828,  0.0000,  5.0376,  ..., 27.6228,  0.0000,  0.0000]])  ||  image_sets/features/F006a06.pt
F006a06
173  ||  [tensor([0.], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.9623,  0.0000, 12.1016,  ..., 26.5228,  0.0000,  0.0000]])  ||  image_sets/

F006a07
207  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.6115,  0.0000,  3.5946,  ..., 24.7278,  0.0000,  0.0000]])  ||  image_sets/features/F006a07.pt
F006a08
208  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.9436,  0.0000,  5.4173,  ..., 25.2008,  0.0000,  0.0000]])  ||  image_sets/features/F006a08.pt
F006a08
209  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.7757,  0.0000,  4.2649,  ..., 29.8245,  0.0000,  0.0000]])  ||  image_sets/features/F006a08.pt
F006a08
210  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.5968,  0.0000,  2.5752,  ..., 34.4299,  0.0000,  0.0000]])  ||  image_sets/features/F006a08.pt
F006a08
211  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.6143,  0.0000,  2.5448,  ..., 33.6674,  0.0000,  0.0000]]) 

...

N005a003
7406  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.6904,  0.0000,  4.2695,  ..., 12.8840,  0.0000,  0.0000]])  ||  image_sets/features/N005a003.pt
N005a003
7407  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.5810,  0.0000,  3.8152,  ..., 15.2874,  0.0000,  0.0000]])  ||  image_sets/features/N005a003.pt
N005a003
7408  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.6314,  0.0000,  3.3803,  ..., 18.1330,  0.0000,  0.0000]])  ||  image_sets/features/N005a003.pt
N005a003
7409  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.8292,  0.0000,  3.8591,  ..., 22.1785,  0.0000,  0.0000]])  ||  image_sets/features/N005a003.pt
N005a004
7410  ||  [tensor([0.0117], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.6572,  0.0000,  1.8088,  ..., 30.9721,  0.000

7477  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.4584,  0.0000,  7.1942,  ..., 26.3816,  0.0000,  0.0000]])  ||  image_sets/features/N005a006.pt
N005a006
7478  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.6205,  0.0000,  8.4609,  ..., 31.1190,  0.0000,  0.0000]])  ||  image_sets/features/N005a006.pt
N005a006
7479  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.7577,  0.0000,  3.5367,  ..., 33.8654,  0.0000,  0.0000]])  ||  image_sets/features/N005a006.pt
N005a006
7480  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.6021,  0.0000,  3.3233,  ..., 25.6232,  0.0000,  0.0000]])  ||  image_sets/features/N005a006.pt
N005a006
7481  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.6343,  0.0000,  2.9329,  ..., 19.5493,  0.0000,  0

N005a009
7548  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.5404,  0.0000,  1.5151,  ..., 46.0195,  0.0000,  0.0000]])  ||  image_sets/features/N005a009.pt
N005a009
7549  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.5235,  0.0000,  1.5246,  ..., 46.3999,  0.0000,  0.0000]])  ||  image_sets/features/N005a009.pt
N005a009
7550  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.5241,  0.0000,  1.6156,  ..., 46.4737,  0.0000,  0.0000]])  ||  image_sets/features/N005a009.pt
N005a009
7551  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.5338,  0.0000,  1.5099,  ..., 45.8174,  0.0000,  0.0000]])  ||  image_sets/features/N005a009.pt
N005a009
7552  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.6231,  0.0000,  2.0289,  ..., 45.7246,  0.000

N005a012
7620  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.4538,  0.0000,  1.2769,  ..., 39.6444,  0.0000,  0.0000]])  ||  image_sets/features/N005a012.pt
N005a012
7621  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.4521,  0.0000,  1.2465,  ..., 39.5426,  0.0000,  0.0000]])  ||  image_sets/features/N005a012.pt
N005a012
7622  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.4906,  0.0000,  1.8560,  ..., 34.5154,  0.0000,  0.0000]])  ||  image_sets/features/N005a012.pt
N005a012
7623  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.5150,  0.0000,  1.3354,  ..., 36.3400,  0.0000,  0.0000]])  ||  image_sets/features/N005a012.pt
N005a012
7624  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0156], dtype=torch.float64)]  ||  tensor([[ 0.4567,  0.0000,  1.3300,  ..., 38.4596,  0.000

7691  ||  [tensor([0.0039], dtype=torch.float64), tensor([0.0195], dtype=torch.float64)]  ||  tensor([[ 0.5039,  0.0000,  1.5226,  ..., 32.8165,  0.0000,  0.0000]])  ||  image_sets/features/N005a014.pt
N005a014
7692  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.], dtype=torch.float64)]  ||  tensor([[ 0.4422,  0.0000,  1.3100,  ..., 34.4871,  0.0000,  0.0000]])  ||  image_sets/features/N005a014.pt
N005a014
7693  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0039], dtype=torch.float64)]  ||  tensor([[ 0.4742,  0.0000,  1.3800,  ..., 34.2562,  0.0000,  0.0000]])  ||  image_sets/features/N005a014.pt
N005a014
7694  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0078], dtype=torch.float64)]  ||  tensor([[ 0.4662,  0.0000,  1.5255,  ..., 34.2749,  0.0000,  0.0000]])  ||  image_sets/features/N005a014.pt
N005a014
7695  ||  [tensor([0.0078], dtype=torch.float64), tensor([0.0117], dtype=torch.float64)]  ||  tensor([[ 0.4526,  0.0000,  1.5591,  ..., 33.5991,  0.0000,  0.000

In [22]:
actual_feat_dir = actual_feat_dir
feat_dir = feat_dir
!mkdir -p $actual_feat_dir
!mv $feat_dir/* $actual_feat_dir